In [103]:
!pip install --upgrade pip
!pip install pandas==1.5.3 seaborn==0.12.2 matplotlib==3.7.1

In [104]:
# Importamos las librerias necesarias.
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [105]:
# Cargamos la data en la variable demografic Data o demoData.
demoData = pd.read_csv('/content/credit_risk_dataset.csv', delimiter=';')

In [106]:
print("La cantidad de filas que tenemos es:", demoData.shape[0],
      "\nY la cantidad de columnas es:", demoData.shape[1],
      "\n\nLas primeras columnas de nuestro data set nos arrojan la siguiente información:")

demoData.head()

La cantidad de filas que tenemos es: 32581 
Y la cantidad de columnas es: 12 

Las primeras columnas de nuestro data set nos arrojan la siguiente información:


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,1230.0,PERSONAL,D,35000,1602.0,1,59,Y,3
1,21,9600,OWN,50.0,EDUCATION,B,1000,1114.0,0,1,N,2
2,25,9600,MORTGAGE,10.0,MEDICAL,C,5500,1287.0,1,57,N,3
3,23,65500,RENT,40.0,MEDICAL,C,35000,1523.0,1,53,N,2
4,24,54400,RENT,80.0,MEDICAL,C,35000,1427.0,1,55,Y,4


In [107]:
demoData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  int64  
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 3.0+ MB


Con lo anterior podemos darnos cuenta que tenemos varios datos faltantes en las siguientes columnas:

- person_emp_length = Duración del empleo (En años)
- loan_int_rate = Tasa de interes

In [108]:
demoData.dtypes

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income             int64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object

Aquí podemos denotar que el data set tiene los tipos de datos de forma correcta de acuerdo al nombre y descripción de las columnas.

In [109]:
demoData.describe().round(1)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.0,32581.0,31686.0,32581.0,29465.0,32581.0,32581.0,32581.0
mean,27.7,66074.8,47.9,9589.4,1044.7,0.2,15.2,5.8
std,6.3,61983.1,41.4,6322.1,397.9,0.4,11.1,4.1
min,20.0,4000.0,0.0,500.0,60.0,0.0,0.0,2.0
25%,23.0,38500.0,20.0,5000.0,751.0,0.0,7.0,3.0
50%,26.0,55000.0,40.0,8000.0,1099.0,0.0,13.0,4.0
75%,30.0,79200.0,70.0,12200.0,1323.0,0.0,22.0,8.0
max,144.0,6000000.0,1230.0,35000.0,2322.0,1.0,83.0,30.0


La tabla anterior la podemos utilizar para identificar valores extraños, por ejemplo en la columna edad, tenemos un valor minimo de edad igual a 20, una media de edad de 27 años y por otro lado tenemos un valor máximo de edad igual a 144, esto no es demasiado lógico así que por ejemplo se trata de valores que podemos eliminar del dataset ya que puede afectar el funcionamiento final del modelo.

In [110]:
demoData = demoData[demoData['person_age'] <= 120]

Dejamos en el dataset exclusivamente personas con una edad inferior a 120 años, esto debido a que la persona viva más longeva actualmente es [Maria Branyas Morera](https://es.wikipedia.org/wiki/Anexo:Personas_vivas_m%C3%A1s_ancianas#:~:text=Desde%20la%20muerte%20de%20la,116%20a%C3%B1os%20y%20180%20d%C3%ADas.)

In [111]:
# Validación de valores nulos
null = demoData.isnull().sum()
nullTable = pd.DataFrame({'Columna': demoData.columns, 'Valores nulos': null})
nullTable = nullTable.reset_index(drop=True)
nullTable

,Columna,Valores nulos
0,person_age,0
1,person_income,0
2,person_home_ownership,0
3,person_emp_length,895
4,loan_intent,0
5,loan_grade,0
6,loan_amnt,0
7,loan_int_rate,3115
8,loan_status,0
9,loan_percent_income,0


Como ya mencionamos anteriormente las columnas de Duración del empleo (En años) y Tasa de interes tienen valores nulos, ambos son valores a tener en cuenta en el modelo así que es importante contar con ellos, por lo tanto vamos a eliminar del dataset cualquier registro que no tenega valor en estos campos.

In [112]:
demoData = demoData[demoData['person_emp_length'].isnull() != True]
demoData = demoData[demoData['loan_int_rate'].isnull() != True]

In [113]:
# Validación de duplicados
duplicates = demoData.duplicated().sum()
duplicatesTable = pd.DataFrame({'Columna': demoData.columns, 'Duplicados': duplicates})
duplicatesTable

,Columna,Duplicados
0,person_age,137
1,person_income,137
2,person_home_ownership,137
3,person_emp_length,137
4,loan_intent,137
5,loan_grade,137
6,loan_amnt,137
7,loan_int_rate,137
8,loan_status,137
9,loan_percent_income,137


Cómo podemos ver tenemos valores duplicados que eliminaremos en la siguiente celda.

In [114]:
demoData = demoData.drop_duplicates()

Ahora vamos a validar nuevamente que ya no tengamos valores atipicos como edades fuera de rango, valores nulos ni valores duplicados.

In [115]:
demoData.describe().round(1)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,28497.0,28497.0,28497.0,28497.0,28497.0,28497.0,28497.0,28497.0
mean,27.7,66434.6,47.9,9658.6,1047.8,0.2,15.2,5.8
std,6.2,51517.3,41.6,6329.6,397.6,0.4,11.0,4.0
min,20.0,4000.0,0.0,500.0,60.0,0.0,0.0,2.0
25%,23.0,39480.0,20.0,5000.0,751.0,0.0,6.0,3.0
50%,26.0,56000.0,40.0,8000.0,1099.0,0.0,13.0,4.0
75%,30.0,80000.0,70.0,12500.0,1323.0,0.0,22.0,8.0
max,84.0,2039784.0,1230.0,35000.0,2322.0,1.0,83.0,30.0


In [116]:
null = demoData.isnull().sum()
nullTable = pd.DataFrame({'Columna': demoData.columns, 'Valores nulos': null})
nullTable = nullTable.reset_index(drop=True)
nullTable

,Columna,Valores nulos
0,person_age,0
1,person_income,0
2,person_home_ownership,0
3,person_emp_length,0
4,loan_intent,0
5,loan_grade,0
6,loan_amnt,0
7,loan_int_rate,0
8,loan_status,0
9,loan_percent_income,0


In [117]:
duplicates = demoData.duplicated().sum()
duplicatesTable = pd.DataFrame({'Columna': demoData.columns, 'Duplicados': duplicates})
duplicatesTable

,Columna,Duplicados
0,person_age,0
1,person_income,0
2,person_home_ownership,0
3,person_emp_length,0
4,loan_intent,0
5,loan_grade,0
6,loan_amnt,0
7,loan_int_rate,0
8,loan_status,0
9,loan_percent_income,0


Ahora vamos a pasar a codificar variables categoricas para que estas puedan ser utilizadas en el modelo.

En este caso utilizamos etiquetado numérico para asignarle un valor unico a cada categoría.

In [118]:
label_encoder = LabelEncoder()
demoData['loan_intent_encoded'] = label_encoder.fit_transform(demoData['loan_intent'])
demoData['person_home_ownership_encoded'] = label_encoder.fit_transform(demoData['person_home_ownership'])
demoData['loan_grade_encoded'] = label_encoder.fit_transform(demoData['loan_grade'])
demoData

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_intent_encoded,person_home_ownership_encoded,loan_grade_encoded
0,22,59000,RENT,1230.0,PERSONAL,D,35000,1602.0,1,59,Y,3,4,3,3
1,21,9600,OWN,50.0,EDUCATION,B,1000,1114.0,0,1,N,2,1,2,1
2,25,9600,MORTGAGE,10.0,MEDICAL,C,5500,1287.0,1,57,N,3,3,0,2
3,23,65500,RENT,40.0,MEDICAL,C,35000,1523.0,1,53,N,2,3,3,2
4,24,54400,RENT,80.0,MEDICAL,C,35000,1427.0,1,55,Y,4,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,10.0,PERSONAL,C,5800,1316.0,0,11,N,30,4,0,2
32577,54,120000,MORTGAGE,40.0,PERSONAL,A,17625,749.0,0,15,N,19,4,0,0
32578,65,76000,RENT,30.0,HOMEIMPROVEMENT,B,35000,1099.0,1,46,N,28,2,3,1
32579,56,150000,MORTGAGE,50.0,PERSONAL,B,15000,1148.0,0,1,N,26,4,0,1


Para la columna "cb_person_default_on_file" tenemos una columna binaria así que vamos a asignarle un valor de 0 o 1:

In [119]:
unique_values = demoData['cb_person_default_on_file'].unique()
print(unique_values)

['Y' 'N']


In [120]:
demoData['cb_person_default_on_file_encoded'] = demoData['cb_person_default_on_file'].replace({'Y': 1, 'N': 0})

In [121]:
demoData

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_intent_encoded,person_home_ownership_encoded,loan_grade_encoded,cb_person_default_on_file_encoded
0,22,59000,RENT,1230.0,PERSONAL,D,35000,1602.0,1,59,Y,3,4,3,3,1
1,21,9600,OWN,50.0,EDUCATION,B,1000,1114.0,0,1,N,2,1,2,1,0
2,25,9600,MORTGAGE,10.0,MEDICAL,C,5500,1287.0,1,57,N,3,3,0,2,0
3,23,65500,RENT,40.0,MEDICAL,C,35000,1523.0,1,53,N,2,3,3,2,0
4,24,54400,RENT,80.0,MEDICAL,C,35000,1427.0,1,55,Y,4,3,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,10.0,PERSONAL,C,5800,1316.0,0,11,N,30,4,0,2,0
32577,54,120000,MORTGAGE,40.0,PERSONAL,A,17625,749.0,0,15,N,19,4,0,0,0
32578,65,76000,RENT,30.0,HOMEIMPROVEMENT,B,35000,1099.0,1,46,N,28,2,3,1,0
32579,56,150000,MORTGAGE,50.0,PERSONAL,B,15000,1148.0,0,1,N,26,4,0,1,0


Ahora vamos a crear un diccionario para cada una de las columnas que fueron codificadas, para posteriormente eliminar las columnas categoricas.

In [122]:
def create_mapping(data, encoded_column, original_column):
    mapping = dict(zip(data[encoded_column], data[original_column]))
    return mapping

loan_intent_mapping = create_mapping(demoData, 'loan_intent_encoded', 'loan_intent')
print(sorted(loan_intent_mapping.items()))

person_home_ownership_mapping = create_mapping(demoData, 'person_home_ownership_encoded', 'person_home_ownership')
print(sorted(person_home_ownership_mapping.items()))

loan_grade_mapping = create_mapping(demoData, 'loan_grade_encoded', 'loan_grade')
print(sorted(loan_grade_mapping.items()))

cb_person_default_on_mapping = create_mapping(demoData, 'cb_person_default_on_file_encoded', 'cb_person_default_on_file')
print(sorted(cb_person_default_on_mapping.items()))

[(0, 'DEBTCONSOLIDATION'), (1, 'EDUCATION'), (2, 'HOMEIMPROVEMENT'), (3, 'MEDICAL'), (4, 'PERSONAL'), (5, 'VENTURE')]
[(0, 'MORTGAGE'), (1, 'OTHER'), (2, 'OWN'), (3, 'RENT')]
[(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E'), (5, 'F'), (6, 'G')]
[(0, 'N'), (1, 'Y')]


Teniendo los diccionarios de nuestras variables categoricas podemos empezar a eliminar las columnas originales.

In [123]:
demoData = demoData.drop(['loan_intent', 'person_home_ownership', 'loan_grade', 'cb_person_default_on_file'], axis=1)
demoData

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,loan_intent_encoded,person_home_ownership_encoded,loan_grade_encoded,cb_person_default_on_file_encoded
0,22,59000,1230.0,35000,1602.0,1,59,3,4,3,3,1
1,21,9600,50.0,1000,1114.0,0,1,2,1,2,1,0
2,25,9600,10.0,5500,1287.0,1,57,3,3,0,2,0
3,23,65500,40.0,35000,1523.0,1,53,2,3,3,2,0
4,24,54400,80.0,35000,1427.0,1,55,4,3,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,10.0,5800,1316.0,0,11,30,4,0,2,0
32577,54,120000,40.0,17625,749.0,0,15,19,4,0,0,0
32578,65,76000,30.0,35000,1099.0,1,46,28,2,3,1,0
32579,56,150000,50.0,15000,1148.0,0,1,26,4,0,1,0


Ahora que tenemos nuestra data tratada vamos a entregar un archivo csv listo para ser utilizado.

In [126]:
demoData.to_csv('dataTratada.csv', index=False)